In [1]:
from unsloth import FastModel
import torch
import torch.nn as nn
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    # model_name = "unsloth/Qwen3-8B-Base-unsloth-bnb-4bit",
    # model_name="unsloth/gemma-3-12b-pt",
    # model_name="unsloth/gemma-3-4b-pt",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 08-03 16:23:50 [__init__.py:235] Automatically detected platform cuda.
==((====))==  Unsloth 2025.7.11: Fast Qwen3 patching. Transformers: 4.54.1. vLLM: 0.10.0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.635 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(2, 4096)
    (layers): ModuleList(
      (0): Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm

In [2]:
# save the lm_head for yes and no
index = [tokenizer.encode(" No")[0], tokenizer.encode(" Yes")[0]]
print(index)
torch.save(model.lm_head.weight[index], './Model/Gwen8B_lm_head.pth')

[2308, 7414]


In [ ]:
from collections import defaultdict
from utility import build_dataloader
import pandas as pd
df = pd.read_csv("Data/Data1/train.csv")
dataloader = build_dataloader(df, tokenizer)
loss_fn = torch.nn.CrossEntropyLoss()
prob_list = defaultdict(list)
for i, (row_id, input_ids, vi_index, labels) in enumerate(dataloader):
    row_id = int(row_id)
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        input_ids, vi_index, labels = input_ids.to('cuda'), vi_index.to('cuda'), labels.to('cuda')
        output = model.model(input_ids)
        logits = output.last_hidden_state[0, vi_index] @ lm_head_weight # (# of Violation, 4096) @ (4096, 2) -> (# of Violation, 2)
        loss = loss_fn(logits[:2], labels) # first 2 tokens are used for training, (N, C), (N,)
        loss.backward()
        if vi_index.shape[0] == 3:
            prob = torch.softmax(logits[2], dim=0)[1].item()
            prob_list[row_id].append(prob)
        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [5]:
prob = torch.softmax(logits[2], dim=0)

In [8]:
prob[1].item()

0.92578125

In [ ]:
loss = loss_fn(logits[:2], labels) # first 2 tokens are used for training, (N, C), (N,)

In [13]:
torch.softmax(logits[2], dim=1)

NameError: name 'logits' is not defined

In [7]:
input_ids[0, 67], input_ids[0, 81]

(tensor(7414, device='cuda:0'), tensor(2308, device='cuda:0'))

In [8]:
labels

tensor([1, 0], device='cuda:0')

In [5]:
input_ids#[0, vi_index+1]

tensor([[ 2610,   525,  2661,   264,  3980,   389, 42174,    13,  4615,  3383,
           374,   311, 48129,   421,   432, 50313,   279,  2661,  5912,   624,
          3136, 36933,    25,   435, 21284, 53344,   198, 11337,    25,  2308,
         46323,    25, 81667,    11, 44780,  7746,    11,  6975, 91104, 12876,
            11,   323, 34282,  2213,   525,   537,  5420,   624, 10677,    25,
          3908, 38788,  3330,   369,   697, 22951,   633,   752,  1588,   366,
          3144,  1670,   757, 61419,   397, 74971,    25,  7414,   198, 10677,
            25,  5145,  1588, 55460,  1784,  3144, 83338,   905,   397, 74971,
            25,  2308,   198, 10677,    25,  5692,   594,   847, 44780,  2656,
            11,  9339,   323,  4669, 35575,   366,  3144,    62,  2136,  7031,
           908,  1327,   802, 24822,   905,   397, 74971,    25]],
       device='cuda:0')

In [23]:
labels

tensor([1, 0], device='cuda:0')

In [24]:
logits[:2]

tensor([[32.2500, 34.2500],
        [35.7500, 37.0000]], device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<SliceBackward0>)

In [20]:
loss

tensor(0.8125, device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<CompiledFunctionBackward>)

In [18]:
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
torch.log(torch.tensor(0.5)) 

tensor(-0.6931)

In [13]:
input_ids.shape

torch.Size([1, 132])

In [16]:
labels

tensor([1, 0], device='cuda:0')